In [5]:
%%writefile bu.cu
#include <iostream>
#include <vector>
#include <cstdlib>  // For rand()
#include <ctime>    // For clock()
#include <sstream>  // For stringstream
#include <string>   // For string

// Sequential Bubble Sort
void bubbleSortSequential(int* arr, int size) {
    for (int i = 0; i < size - 1; i++) {
        for (int j = 0; j < size - i - 1; j++) {
            if (arr[j] > arr[j + 1]) {
                int temp = arr[j];
                arr[j] = arr[j + 1];
                arr[j + 1] = temp;
            }
        }
    }
}

// Parallel Bubble Sort using CUDA
__global__ void bubbleSortParallel(int* arr, int size) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    if (tid < size) {
        for (int i = 0; i < size - 1; i++) {
            for (int j = 0; j < size - i - 1; j++) {
                if (arr[j] > arr[j + 1]) {
                    int temp = arr[j];
                    arr[j] = arr[j + 1];
                    arr[j + 1] = temp;
                }
            }
        }
    }
}

int main() {
    int size;
    std::cout << "Enter the number of elements: ";
    std::cin >> size;

    // Input elements from the user
    std::cout << "Enter the elements separated by spaces: ";
    std::string input;
    std::getline(std::cin >> std::ws, input);

    std::vector<int> elements;
    std::stringstream ss(input);
    int num;
    while (ss >> num) {
        elements.push_back(num);
    }

    // Convert vector to array
    int* arr = new int[size];
    for (int i = 0; i < size; ++i) {
        arr[i] = elements[i];
    }

    // Measure sequential bubble sort time
    clock_t startSeqBubble = clock();
    bubbleSortSequential(arr, size);
    clock_t endSeqBubble = clock();
    double timeSeqBubble = double(endSeqBubble - startSeqBubble) / CLOCKS_PER_SEC;

    // Measure parallel bubble sort time
    int* d_arr;
    cudaMalloc(&d_arr, size * sizeof(int));
    cudaMemcpy(d_arr, arr, size * sizeof(int), cudaMemcpyHostToDevice);

    clock_t startParBubble = clock();
    bubbleSortParallel<<<(size + 255) / 256, 256>>>(d_arr, size);
    cudaDeviceSynchronize();
    clock_t endParBubble = clock();
    double timeParBubble = double(endParBubble - startParBubble) / CLOCKS_PER_SEC;

    // Retrieve sorted array from device memory
    cudaMemcpy(arr, d_arr, size * sizeof(int), cudaMemcpyDeviceToHost);

    // Print execution times
    std::cout << "Sequential Bubble Sort Time: " << timeSeqBubble << " seconds" << std::endl;
    std::cout << "Parallel Bubble Sort Time: " << timeParBubble << " seconds" << std::endl;

    // Print sorted array (only a few elements for brevity)
    std::cout << "Sorted Array (only showing first 10 elements): ";
    for (int i = 0; i < std::min(size, 10); ++i) {
        std::cout << arr[i] << " ";
    }
    std::cout << std::endl;

    // Free device memory
    cudaFree(d_arr);

    delete[] arr;

    return 0;
}


Overwriting bu.cu


In [6]:
!nvcc bu.cu -o bu
!./bu

Enter the number of elements: 5
Enter the elements separated by spaces: 3 4 8 7 1
Sequential Bubble Sort Time: 2e-06 seconds
Parallel Bubble Sort Time: 4e-06 seconds
Sorted Array (only showing first 10 elements): 1 3 4 7 8 


In [18]:
%%writefile merge.cu

#include <iostream>
#include <vector>
#include <chrono>
#include <cuda_runtime.h>

// CUDA kernel for Bubble Sort
__global__ void bubble_sort(int* d_arr, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x; // Thread index
    for (int i = 0; i < size - 1; i++) {
        int j = idx + i; // Offset to perform the bubble sort step
        if (j < size - 1 && d_arr[j] > d_arr[j + 1]) { // Swap if out of order
            int temp = d_arr[j];
            d_arr[j] = d_arr[j + 1];
            d_arr[j + 1] = temp;
        }
        __syncthreads(); // Synchronize threads within block
    }
}

// Function for Bubble Sort on CPU
void bubble_sort_cpu(int* arr, int size) {
    for (int i = 0; i < size - 1; i++) {
        for (int j = 0; j < size - 1 - i; j++) {
            if (arr[j] > arr[j + 1]) { // Swap if out of order
                int temp = arr[j];
                arr[j] = arr[j + 1];
                arr[j + 1] = temp;
            }
        }
    }
}

int main() {
    // Test data
    int h_arr[] = {64, 34, 25, 12, 22, 11, 90};
    int size = sizeof(h_arr) / sizeof(h_arr[0]);

    // Bubble Sort on CPU
    auto start = std::chrono::high_resolution_clock::now();
    bubble_sort_cpu(h_arr, size);
    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> duration = end - start;
    std::cout << "Sequential Merge Sort took " << duration.count() << " seconds\n";

    // Copying data to the device for parallel Bubble Sort
    int* d_arr;
    cudaMalloc(&d_arr, size * sizeof(int));
    cudaMemcpy(d_arr, h_arr, size * sizeof(int), cudaMemcpyHostToDevice);

    // Bubble Sort on GPU
    start = std::chrono::high_resolution_clock::now();
    int blockSize = 256; // Threads per block
    int gridSize = (size + blockSize - 1) / blockSize; // Blocks
    bubble_sort<<<gridSize, blockSize>>>(d_arr, size);
    cudaMemcpy(h_arr, d_arr, size * sizeof(int), cudaMemcpyDeviceToHost); // Copy back to host
    end = std::chrono::high_resolution_clock::now();
    duration = end - start;
    std::cout << "Parallel Merge Sort took " << duration.count() << " seconds\n";

    // Display sorted array
    std::cout << "Sorted Array: ";
    for (int i = 0; i < size; i++) {
        std::cout << h_arr[i] << " ";
    }
    std::cout << std::endl;

    // Free device memory
    cudaFree(d_arr);

    return 0;
}

Writing merge.cu


In [21]:
!nvcc merge.cu -o merge
!./merge

Sequential Merge Sort took 5.18e-07 seconds
Parallel Merge Sort took 2.189e-06 seconds
Sorted Array: 11 12 22 25 34 64 90 
